In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.evaluate.basics import eval_model, read_saved_epochs
from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, match_incomplete_to_complete_data
from tf_lab.autopredictors.plotting.basics import plot_original_pclouds_vs_reconstructed,\
                                                  plot_train_val_test_curves, plot_reconstructions_at_epoch




from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from geo_tool import Point_Cloud

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
class_type = raw_input('Give me the class type.\n').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]

experiment_name = '1_2_fc_decoder_1024pts'
do_training = True

seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/1024/', class_syn_id)

train_dir = osp.join(top_data_dir, 'OUT/models/vanilla_ae/', class_syn_id)
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

Give me the class type.
chair


'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/vanilla_ae/03001627/1_2_fc_decoder_1024pts'

In [4]:
full_file_names = pio.load_filenames_of_input_data(full_pclouds_path)
full_pclouds, full_model_names, full_syn_ids = pio.load_crude_point_clouds(file_names=full_file_names, n_threads=15)
print '%d files containing complete point clouds were found.' % (len(full_pclouds), )

6778 files containing complete point clouds were found.


In [5]:
train_data_, val_data_, test_data_, r_indices = pio.train_validate_test_split([full_pclouds, 
                                                                    full_model_names],
                                                                    train_perc=0.85,
                                                                    validate_perc=0.05,
                                                                    test_perc=0.1,                                                                   
                                                                    seed=seed)

model_ids = np.vstack((full_model_names, full_syn_ids)).T
write_model_ids_of_datasets(train_dir, model_ids, r_indices)

train_data = PointCloudDataSet(train_data_[0], labels=train_data_[1])
val_data = PointCloudDataSet(val_data_[0], labels=val_data_[1])
test_data = PointCloudDataSet(test_data_[0], labels=test_data_[1])

In [7]:
n_input = [1024, 3]

decoder_args = {'layer_sizes': [1024, np.prod(n_input)], 
                'non_linearity': tf.nn.relu
               }
 
conf = Conf(
            n_input = n_input,
            training_epochs = 1000,
            batch_size = 50,
            loss = 'chamfer',
            train_dir = train_dir,
            loss_display_step = 1,
            saver_step = 10,
            learning_rate = 0.00002,
            saver_max_to_keep = 100,
            gauss_augment = {'mu': 0, 'sigma': 0.02},            
            encoder = enc_dec.encoder_with_convs_and_symmetry,
            decoder = enc_dec.decoder_with_fc_only,
            decoder_args = decoder_args,
           )

conf.experiment_name = experiment_name
conf.save(osp.join(conf.train_dir, 'configuration'))

In [8]:
reset_tf_graph()
set_visible_GPUs([0])
ae = PointNetAutoEncoder(experiment_name, conf)

if do_training:
    ae.train(train_data, conf)

else:
    saved_epochs = read_saved_epochs(conf.train_dir)
    ae.restore_model(train_dir, saved_epochs[-1])

('Epoch:', '0001', 'training time (minutes)=', '0.1171', 'loss=', '0.005617352')
('Epoch:', '0002', 'training time (minutes)=', '0.1025', 'loss=', '0.003413859')
('Epoch:', '0003', 'training time (minutes)=', '0.1123', 'loss=', '0.003118270')
('Epoch:', '0004', 'training time (minutes)=', '0.1024', 'loss=', '0.002879316')
('Epoch:', '0005', 'training time (minutes)=', '0.1009', 'loss=', '0.002710940')
('Epoch:', '0006', 'training time (minutes)=', '0.1026', 'loss=', '0.002587657')
('Epoch:', '0007', 'training time (minutes)=', '0.1015', 'loss=', '0.002479747')
('Epoch:', '0008', 'training time (minutes)=', '0.1058', 'loss=', '0.002388066')
('Epoch:', '0009', 'training time (minutes)=', '0.1067', 'loss=', '0.002308613')
('Epoch:', '0010', 'training time (minutes)=', '0.1024', 'loss=', '0.002238860')
('Epoch:', '0011', 'training time (minutes)=', '0.1070', 'loss=', '0.002179914')
('Epoch:', '0012', 'training time (minutes)=', '0.1075', 'loss=', '0.002126225')
('Epoch:', '0013', 'training

In [ ]:
# plot_epochs = saved_epochs[::len(saved_epochs)/10]
# stats = eval_model(vae, conf, [train_data, val_data, test_data], epochs=plot_epochs, verbose=True)
# top_save_dir = osp.join(top_data_dir, 'OUT/model_evaluation/?/', experiment_name)
# plot_train_val_test_curves(stats, top_save_dir, has_validation=True, best_epoch=None, show=True)